In [ ]:
import random
from __future__ import division
import math
import numpy as np
import copy
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class RandomWalkEnv(object):
    def __init__(self, num_states, hop, start_state=500):
        self.num_states = num_states
        self.hop = hop
        self.state = None
        self.reset(start_state)
        
    def reset(self, start_state):
        self.state = start_state
        
    def get_reward(self):
        reward = 0
        done = False
        if self.state == 0:
            reward = -1
            done = True
        elif self.state == self.num_states-1:
            reward = 1
            done = True
        return reward, done
        
    def step(self):
        displacement = random.randint(1, self.hop)
        if (random.random() > 0.5):
            displacement *= -1
        self.state += displacement 
        self.state = min(self.num_states-1, max(0, self.state))        
        reward, done = self.get_reward()
        return reward, self.state, done
    
    def get_episode(self, start_state):
        self.reset(start_state)
        done = False
        episode = [(None, start_state)]
        while not done:
            reward, state, done = self.step()
            episode.append((reward, state))
        return episode

In [ ]:
def get_returns(episode, gamma):
    Gt = 0
    returns = []
    for t in reversed(range(len(episode)-1)):
        state = episode[t][1]
        reward = episode[t+1][0]
        Gt = gamma*Gt + reward
        returns.append((state, Gt))
    returns.reverse()
    return returns

In [ ]:
def update_state_distribution(distribution, episode):
    states = zip(*episode)[1]
    for state in states:
        distribution[state] += 1
    return distribution

In [ ]:
def get_distribution(episodes, num_states):
    distribution = np.zeros(num_states)
    for episode in episodes:
        distribution = update_state_distribution(distribution, episode)
    total = sum([len(episode) - 1 for episode in episodes])
    return distribution[1:-1] / total

In [ ]:
def get_stopping_times(episodes):
    stopping_times = []
    for episode in episodes:
        stopping_times.append(len(episode) - 1)
    return stopping_times

In [ ]:
def v_agg(state, theta, agg_stride, absorbing_states):
    v = 0
    if state not in absorbing_states:
        bin_num = math.floor((state - 1) / agg_stride)    
        v = theta[bin_num]
    return v

def grad_v_agg(state, theta, agg_stride, absorbing_states):
    grad = np.zeros_like(theta)
    if state not in absorbing_states:
        bin_num = math.floor((state - 1) / agg_stride)        
        grad[bin_num] = 1
    return grad

In [ ]:
def init_theta(num_states, expts):
    theta, theta_hist = {}, {}
    for expt in expts:
        expt_id, agg_stride, _, _, _ = expt
        num_bins = int(math.ceil((num_states - 2) / agg_stride))
        theta[expt_id] = np.zeros(num_bins)
        theta_hist[expt_id] = [copy.deepcopy(theta[expt_id])]
    return theta, theta_hist

In [ ]:
def mc_approx(num_states, episodes, expts, gamma):
    theta, theta_hist = init_theta(num_states, expts)
    for episode_num, episode in enumerate(episodes):
        if episode_num % 1000 == 0:
            print 'Episode number {0}'.format(episode_num)
        returns = get_returns(episode, gamma)
        for state, Gt in returns:
            for expt in expts:
                expt_id, _, alpha, v_agg_expt, grad_v_agg_expt = expt
                v_estimate = v_agg_expt(state, theta[expt_id])
                grad_v = grad_v_agg_expt(state, theta[expt_id])
                theta[expt_id] += alpha*(Gt - v_estimate)*grad_v
        for expt in expts:
            expt_id, _, _, _, _ = expt
            theta_hist[expt_id].append(copy.deepcopy(theta[expt_id]))
    return theta_hist

In [ ]:
def semi_gradient_td0(num_states, episodes, expts, gamma):
    theta, theta_hist = init_theta(num_states, expts)
    for episode_num, episode in enumerate(episodes):
        if episode_num % 1000 == 0:
            print 'Episode number {0}'.format(episode_num)
        for t in range(len(episode)-1):
            s = episode[t][1]
            r, s_prime = episode[t+1]
            for expt in expts:
                expt_id, _, alpha, v_agg_expt, grad_v_agg_expt = expt
                v_s = v_agg_expt(s, theta[expt_id])
                v_s_prime = v_agg_expt(s_prime, theta[expt_id])
                grad_v_s = grad_v_agg_expt(s, theta[expt_id])
                theta[expt_id] += alpha*(r + gamma*v_s_prime - v_s)*grad_v_s
        for expt in expts:
            expt_id, _, _, _, _ = expt
            theta_hist[expt_id].append(copy.deepcopy(theta[expt_id]))
    return theta_hist

In [ ]:
def demi_gradient_td0(num_states, episodes, expts, gamma, delta):
    theta, theta_hist = init_theta(num_states, expts)
    for episode_num, episode in enumerate(episodes):
        if episode_num % 1000 == 0:
            print 'Episode number {0}'.format(episode_num)
        for t in range(len(episode)-1):
            s = episode[t][1]
            r, s_prime = episode[t+1]
            for expt in expts:
                expt_id, _, alpha, v_agg_expt, grad_v_agg_expt = expt
                v_s = v_agg_expt(s, theta[expt_id])
                v_s_prime = v_agg_expt(s_prime, theta[expt_id])
                grad_v_s = grad_v_agg_expt(s, theta[expt_id])
                grad_v_s_prime = grad_v_agg_expt(s_prime, theta[expt_id])
                theta[expt_id] += alpha*(r + gamma*v_s_prime - v_s)*(grad_v_s - delta*gamma*grad_v_s_prime)
        for expt in expts:
            expt_id, _, _, _, _ = expt
            theta_hist[expt_id].append(copy.deepcopy(theta[expt_id]))
    return theta_hist

In [ ]:
def viz_theta(expts, theta_hist, i):
    plt.figure(figsize=[10, 5])
    for expt in expts:
        (expt_id, agg_stride, alpha, v_agg_expt, grad_v_agg_expt) = expt
        temp = np.tile(theta_hist[expt_id][i], [agg_stride, 1])
        temp = np.reshape(temp.T, -1)
        plt.plot(temp, alpha=0.7)
    plt.ylim([-1, 1])
    plt.xlabel('state')
    plt.ylabel('value')

In [ ]:
def spec2expt(spec, num_states):
    absorbing_states = [0, num_states-1]
    expt_id, agg_stride, alpha = spec
    v_agg_expt = lambda state, theta : v_agg(state, theta, agg_stride, absorbing_states)
    grad_v_agg_expt = lambda state, theta : grad_v_agg(state, theta, agg_stride, absorbing_states)    
    expt = (expt_id, agg_stride, alpha, v_agg_expt, grad_v_agg_expt)
    return expt

In [ ]:
def gen_episodes(num_states, hop, start_state, num_iter):
    random_walk_env = RandomWalkEnv(num_states, hop, start_state=start_state)
    episodes = [random_walk_env.get_episode(start_state) for _ in range(num_iter)]
    return episodes

In [ ]:
# Generate some data
num_states = 1002
hop = 100
start_state = 500
num_iter = 1000
episodes = gen_episodes(num_states, hop, start_state, num_iter)

In [ ]:
# Plot 1
stopping_times = get_stopping_times(episodes)
plt.figure(figsize=[10, 5])
plt.hist(stopping_times, 100)
plt.xlabel('number of steps')
plt.ylabel('frequency')
plt.show()

In [ ]:
# Plot 2
distribution = get_distribution(episodes, num_states)
plt.figure(figsize=[10, 5])
plt.plot(distribution)
plt.xlabel('state')
plt.ylabel('frequency')
plt.show()

In [ ]:
# Plot 3
spec1 = ('agg100', 100, 2e-5)
spec2 = ('full', 1, 1e-3)
specs = [spec1, spec2]
expts = [spec2expt(spec, num_states) for spec in specs]
theta_hist_mc = mc_approx(num_states, episodes, expts, gamma=1)
i_slider = widgets.IntSlider(min=0, max=num_iter, step=1, value=num_iter)
f1 = lambda i: viz_theta(expts, theta_hist_mc, i)
interact(f1, i=i_slider)

In [ ]:
# Plot 4
spec1 = ('agg10', 100, 2e-5*100)
spec2 = ('full', 1, 1e-3*100)
specs = [spec1, spec2]
expts = [spec2expt(spec, num_states) for spec in specs]
theta_hist_semi = semi_gradient_td0(num_states, episodes, expts, gamma=1)
i_slider = widgets.IntSlider(min=0, max=num_iter, step=1, value=num_iter)
f1 = lambda i: viz_theta(expts, theta_hist_semi, i)
interact(f1, i=i_slider)

In [ ]:
# Plot 5
spec1 = ('agg100', 100, 2e-5*200)
spec2 = ('full', 1, 1e-3*200)
specs = [spec1, spec2]
expts = [spec2expt(spec, num_states) for spec in specs]
theta_hist_demi = demi_gradient_td0(num_states, episodes, expts, gamma=1, delta=1.0)
i_slider = widgets.IntSlider(min=0, max=num_iter, step=1, value=num_iter)
f1 = lambda i: viz_theta(expts, theta_hist_demi, i)
interact(f1, i=i_slider)

In [ ]:
# Generate smaller version of data
num_states = 12
hop = 1
start_state = 5
num_iter = 1000
episodes = gen_episodes(num_states, hop, start_state, num_iter)

In [ ]:
theta_star = (2*(np.arange(num_states) / (num_states - 1)) - 1)[1:-1]
# print theta_star
# plt.plot(theta_star)

In [ ]:
# spec1 = ('agg2', 2, 2e-5)
spec2 = ('full', 1, 1e-3)
specs = [spec2]
expts = [spec2expt(spec, num_states) for spec in specs]
theta_hist_mc = mc_approx(num_states, episodes, expts, gamma=1)
error_mc = [np.linalg.norm(theta - theta_star) for theta in theta_hist_mc['full']]

spec2 = ('full', 1, 1e-3*10)
specs = [spec2]
expts = [spec2expt(spec, num_states) for spec in specs]
theta_hist_semi = semi_gradient_td0(num_states, episodes, expts, gamma=1)
error_semi = [np.linalg.norm(theta - theta_star) for theta in theta_hist_semi['full']]

spec2 = ('full', 1, 1e-3*10)
specs = [spec2]
expts = [spec2expt(spec, num_states) for spec in specs]
deltas = [-0.4, -0.1, 0.3, 0.7, 1]
error_demis = []
for delta in deltas:
    theta_hist_demi = demi_gradient_td0(num_states, episodes, expts, gamma=1, delta=delta)
    error_demi = [np.linalg.norm(theta - theta_star) for theta in theta_hist_demi['full']]
    error_demis.append(error_demi)

In [ ]:
plt.figure(figsize=[10, 5])
plt.plot(error_mc)
plt.plot(error_semi)
for error_demi in error_demis:
    plt.plot(error_demi)
legends = ['monte carlo', 'semi gradient']
for delta in deltas:
    legends.append('delta = {0}'.format(delta))
plt.legend(legends)
plt.xlabel('episodes')
plt.ylabel('RMS error')